In [1]:
# No Parameters as settings obtained dynamically from default spark lakehouse


StatementMeta(, 85fc75e7-cf27-4ed4-93e9-e1cc206c8b79, 3, Finished, Available, Finished)

In [2]:
# Fabric Refresh Logs
import pandas as pd
from delta.tables import *
import sempy.fabric as fabric
import re
from pyspark.sql.types import StructType, StructField,  LongType, StringType,DateType, TimestampType,MapType

tenant_id=spark.conf.get("trident.tenant.id")
workspace_id=spark.conf.get("trident.workspace.id")
lakehouse_id=spark.conf.get("trident.lakehouse.id")
lakehouse_name=spark.conf.get("trident.lakehouse.name")
column_pattern = '[ ,;{}()\n\t/=]' #Pattern to remove invalid columns for lakehouse

# List Datasets/SemanticModels
df_datasets =fabric.list_datasets()

table_name="fabric_refresh_logs"
for row in df_datasets.itertuples(index=True, name='datasets'):
    dataset = row[1]
    df=fabric.list_refresh_requests(dataset=dataset, workspace=workspace_id, top_n=100)
    df=df.rename(columns=dict(zip(df.columns, [re.sub(column_pattern, '_', col.strip(column_pattern).lower()) for col in df.columns])))
    df.insert(0, 'dataset', dataset)
    df.insert(1, 'workspace', workspace_id)
    df['refresh_attempts'] = df['refresh_attempts'].astype(str) 
    df.drop(columns=['extended_status'])

    schema = StructType([
    StructField("dataset", StringType(), True),
    StructField("workspace", StringType(), True),
    StructField("id",  LongType(), True),
    StructField("request_id", StringType(), True),
    StructField("start_time", TimestampType(), True),
    StructField("end_time", TimestampType(), True),
    StructField("refresh_type", StringType(), True),
    StructField("service_exception_json", StringType(), True),
    StructField("status", StringType(), True),
    StructField("refresh_attempts", StringType(), True)
    ])

    spark_df =spark.createDataFrame(df,schema =schema )
    if spark.catalog.tableExists(table_name):
        target_table = DeltaTable.forName(spark, f"{table_name}")
        target_table.alias("target").merge(spark_df.alias("source"), "source.id=target.id").whenNotMatchedInsertAll().execute()
    else:
        spark_df.write.mode('overwrite').saveAsTable(table_name)


StatementMeta(, 85fc75e7-cf27-4ed4-93e9-e1cc206c8b79, 4, Finished, Available, Finished)